In [1]:
#Data Manipulation
import numpy as np
import pandas as pd

#List of countries
import pycountry

#Data Visualization
import matplotlib.pyplot as plt
import plotly.express as px

#Time Series
from statsmodels.tsa.arima.model import ARIMA
import pmdarima as pm

# Preprocessing

In [2]:
# Load the Dataset
# The source of the dataset is the United Nations (UN)
# It has historical population data from 1950 -2020 and predicted data from 2021-2100

df_pop = pd.read_csv('../raw_data/WPP2019_TotalPopulationBySex_Original.csv', sep=";", decimal=",")
df_pop.tail(2)

,LocID,Location,VarID,Variant,Time,MidPeriod,PopMale,PopFemale,PopTotal,PopDensity
280930,716,Zimbabwe,207,Lower 95 PI,2095,2095.5,9503.711,10412.184,19892.080,51.421
280931,716,Zimbabwe,207,Lower 95 PI,2100,2100.5,9090.075,9996.105,19061.177,49.273


In [3]:
# Confirming that the data type of each column is the correct one

df_pop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 280932 entries, 0 to 280931
Data columns (total 10 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   LocID       280932 non-null  int64  
 1   Location    280932 non-null  object 
 2   VarID       280932 non-null  int64  
 3   Variant     280932 non-null  object 
 4   Time        280932 non-null  int64  
 5   MidPeriod   280932 non-null  float64
 6   PopMale     250876 non-null  float64
 7   PopFemale   250876 non-null  float64
 8   PopTotal    280932 non-null  float64
 9   PopDensity  280932 non-null  float64
dtypes: float64(5), int64(3), object(2)
memory usage: 21.4+ MB


In [4]:
# Select only 1 variant (fourth column) for our predictions
# Accorging to the UN, the "Medium" variant is the most used

variants = df_pop['Variant'].unique()
for count, value in enumerate(variants):
    print(count, value)

0 Medium
1 High
2 Low
3 Constant fertility
4 Instant replacement
5 Zero migration
6 Constant mortality
7 No change
8 Momentum
9 Median PI
10 Upper 80 PI
11 Lower 80 PI
12 Upper 95 PI
13 Lower 95 PI


In [5]:
#Filtering the DataFrame to Variant "Medium"

df_pop_medium = df_pop[df_pop['Variant']=='Medium']
df_pop_medium.tail(2)

,LocID,Location,VarID,Variant,Time,MidPeriod,PopMale,PopFemale,PopTotal,PopDensity
280197,716,Zimbabwe,2,Medium,2099,2099.5,15004.963,15959.089,30964.052,80.041
280198,716,Zimbabwe,2,Medium,2100,2100.5,15001.252,15964.169,30965.421,80.045


In [6]:
# Filtering the DataFrame so it only has the historical data, that is 1950 -2020

df_pop_medium.drop(df_pop_medium.loc[df_pop_medium['Time']>2020].index, inplace=True)
df_pop_medium.tail(2)

/var/folders/k_/r7lt0qq16rjb59h6p_fphwnm0000gn/T/ipykernel_15432/105989058.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pop_medium.drop(df_pop_medium.loc[df_pop_medium['Time']>2020].index, inplace=True)


,LocID,Location,VarID,Variant,Time,MidPeriod,PopMale,PopFemale,PopTotal,PopDensity
280117,716,Zimbabwe,2,Medium,2019,2019.5,6983.353,7662.120,14645.473,37.858
280118,716,Zimbabwe,2,Medium,2020,2020.5,7092.010,7770.917,14862.927,38.420


In [7]:
# Reset index

df_pop_medium = df_pop_medium.reset_index(drop=True)
df_pop_medium

,LocID,Location,VarID,Variant,Time,MidPeriod,PopMale,PopFemale,PopTotal,PopDensity
0,4,Afghanistan,2,Medium,1950,1950.5,4099.243,3652.874,7752.117,11.874
1,4,Afghanistan,2,Medium,1951,1951.5,4134.756,3705.395,7840.151,12.009
2,4,Afghanistan,2,Medium,1952,1952.5,4174.450,3761.546,7935.996,12.156
3,4,Afghanistan,2,Medium,1953,1953.5,4218.336,3821.348,8039.684,12.315
4,4,Afghanistan,2,Medium,1954,1954.5,4266.484,3884.832,8151.316,12.486
...,...,...,...,...,...,...,...,...,...,...
33862,716,Zimbabwe,2,Medium,2016,2016.5,6674.206,7356.132,14030.338,36.268
33863,716,Zimbabwe,2,Medium,2017,2017.5,6777.054,7459.545,14236.599,36.801
33864,716,Zimbabwe,2,Medium,2018,2018.5,6879.119,7559.693,14438.812,37.324
33865,716,Zimbabwe,2,Medium,2019,2019.5,6983.353,7662.120,14645.473,37.858


In [8]:
#Checking for missing values for PopTotal field

df_pop_medium.isna().sum()

LocID            0
Location         0
VarID            0
Variant          0
Time             0
MidPeriod        0
PopMale       2414
PopFemale     2414
PopTotal         0
PopDensity       0
dtype: int64

In [9]:
# Checking for duplicates

df_pop_medium.duplicated().sum()

0

## Defining the list of countries

In [10]:
# Creating a list with all the unique locations on the "Location" field.
# Our data set includes not only countries, but economic alliances, continents, etc..

locations = df_pop['Location'].unique()
locations =  list(locations)
len(locations)

474

In [11]:
# Creating a list with all the countries using pycountry

list_countries = []

def get_countries():
    for country in pycountry.countries:
        list_countries.append(country.name)
        
get_countries()
len(list_countries)

249

In [12]:
# Comparing both lists and creating a new list with the ones that match

len(set(locations).intersection(list_countries))

215

In [14]:
# Some countries are missing, due to the way they were written. 
# E.g: 'Bolivia (Plurinational State of)' vs 'Bolivia, Plurinational State of'
# We will rename the countries of the 'list_countries' list to match the 'locations' list spelling.
# For this, first we identify the index of each country name to be updated

for count, value in enumerate(list_countries):
    print(count, value)

0 Aruba
1 Afghanistan
2 Angola
3 Anguilla
4 Åland Islands
5 Albania
6 Andorra
7 United Arab Emirates
8 Argentina
9 Armenia
10 American Samoa
11 Antarctica
12 French Southern Territories
13 Antigua and Barbuda
14 Australia
15 Austria
16 Azerbaijan
17 Burundi
18 Belgium
19 Benin
20 Bonaire, Sint Eustatius and Saba
21 Burkina Faso
22 Bangladesh
23 Bulgaria
24 Bahrain
25 Bahamas
26 Bosnia and Herzegovina
27 Saint Barthélemy
28 Belarus
29 Belize
30 Bermuda
31 Bolivia, Plurinational State of
32 Brazil
33 Barbados
34 Brunei Darussalam
35 Bhutan
36 Bouvet Island
37 Botswana
38 Central African Republic
39 Canada
40 Cocos (Keeling) Islands
41 Switzerland
42 Chile
43 China
44 Côte d'Ivoire
45 Cameroon
46 Congo, The Democratic Republic of the
47 Congo
48 Cook Islands
49 Colombia
50 Comoros
51 Cabo Verde
52 Costa Rica
53 Cuba
54 Curaçao
55 Christmas Island
56 Cayman Islands
57 Cyprus
58 Czechia
59 Germany
60 Djibouti
61 Dominica
62 Denmark
63 Dominican Republic
64 Algeria
65 Ecuador
66 Egypt
67 Eri

In [15]:
list_countries[31] = 'Bolivia (Plurinational State of)'
list_countries[181] = "Dem. People's Republic of Korea"
list_countries[46] = 'Democratic Republic of the Congo'
list_countries[236] = 'Holy See'
list_countries[107] = 'Iran (Islamic Republic of)'
list_countries[184] = 'State of Palestine'
list_countries[122] = 'Republic of Korea'
list_countries[229] = 'United Republic of Tanzania'
list_countries[234] = 'United States of America'
list_countries[238] = 'Venezuela (Bolivarian Republic of)'

In [16]:
# We have 10 additional names in our list, which maches the names we updated! 

final_country_list = list(set(locations).intersection(list_countries))
len(final_country_list)

225

In [18]:
# We sort the list 

final_country_list = sorted(final_country_list)
len(final_country_list)

225

# Forecast Total Population Per Country

## Function Definition

In [19]:
def prediction(df, country_list, start_year, end_year):
    '''Makes forecasts using ARIMA method, returns a dictionary with per country predictions'''
    
    #Creating empty dictionary where the forecast will be added
    forecasts_dict = {}
    
    #Looping thought the countries list to apply the model
    for i in country_list:
        df_pred = df[df.Location==i][['Time','PopTotal']]
        df_pred.set_index('Time', inplace= True)
        smodel = pm.auto_arima(df_pred, 
                           seasonal=False, 
                           start_p=0, 
                           max_p=4, 
                           max_d=3, 
                           start_q=0, 
                           max_q=4,
                           trace=True,
                           error_action='ignore')
        
        #1. Initialize model
        order = smodel.get_params()['order']
        arima = ARIMA(df_pred, order=order)
        
        #2. Fit the model
        arima = arima.fit()
        forecast = arima.forecast((end_year-start_year), alpha=0.05)
        
        # 3. Adding forecast per country to the dictionary
        forecasts_dict[i] = forecast
        
        #4. Transformig the dictionary to a DataFrame
        #df_forecasts=pd.DataFrame.from_dict(forecasts_dict)
        df_forecasts=pd.DataFrame(forecasts_dict)
        
        #5. Adding the year of the prediction as the index
        df_forecasts.index = range(start_year, end_year, 1)
    
    return df_forecasts

In [20]:
def to_csv(df, file_name):
    '''Transforms DataFrame to csv file'''

    df.T.to_csv(f'../results/{file_name}.csv', sep=";", decimal=",")

## Making Predictions

### Prediction Per Country (2021-2050)

In [25]:
%%timeit

final_result = prediction(df_pop_medium, final_country_list, 2021, 2051)

Performing stepwise search to minimize aic
 ARIMA(0,2,0)(0,0,0)[0] intercept   : AIC=854.366, Time=0.01 sec
 ARIMA(1,2,0)(0,0,0)[0] intercept   : AIC=767.511, Time=0.03 sec
 ARIMA(0,2,1)(0,0,0)[0] intercept   : AIC=779.497, Time=0.05 sec
 ARIMA(0,2,0)(0,0,0)[0]             : AIC=853.064, Time=0.00 sec
 ARIMA(2,2,0)(0,0,0)[0] intercept   : AIC=639.048, Time=0.03 sec
 ARIMA(3,2,0)(0,0,0)[0] intercept   : AIC=626.298, Time=0.05 sec
 ARIMA(4,2,0)(0,0,0)[0] intercept   : AIC=627.662, Time=0.09 sec
 ARIMA(3,2,1)(0,0,0)[0] intercept   : AIC=641.955, Time=0.15 sec
 ARIMA(2,2,1)(0,0,0)[0] intercept   : AIC=631.186, Time=0.05 sec
 ARIMA(4,2,1)(0,0,0)[0] intercept   : AIC=620.164, Time=0.14 sec
 ARIMA(4,2,2)(0,0,0)[0] intercept   : AIC=inf, Time=0.19 sec
 ARIMA(3,2,2)(0,0,0)[0] intercept   : AIC=629.527, Time=0.07 sec
 ARIMA(4,2,1)(0,0,0)[0]             : AIC=618.802, Time=0.08 sec
 ARIMA(3,2,1)(0,0,0)[0]             : AIC=641.809, Time=0.08 sec
 ARIMA(4,2,0)(0,0,0)[0]             : AIC=626.284, 

/Users/isi/.pyenv/versions/3.9.9/envs/population-prediction/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/isi/.pyenv/versions/3.9.9/envs/population-prediction/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/isi/.pyenv/versions/3.9.9/envs/population-prediction/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/isi/.pyenv/versions/3.9.9/envs/population-prediction/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.

 ARIMA(0,2,1)(0,0,0)[0] intercept   : AIC=419.519, Time=0.03 sec
 ARIMA(0,2,0)(0,0,0)[0]             : AIC=493.225, Time=0.01 sec
 ARIMA(2,2,0)(0,0,0)[0] intercept   : AIC=287.927, Time=0.03 sec
 ARIMA(3,2,0)(0,0,0)[0] intercept   : AIC=275.241, Time=0.04 sec
 ARIMA(4,2,0)(0,0,0)[0] intercept   : AIC=277.239, Time=0.04 sec
 ARIMA(3,2,1)(0,0,0)[0] intercept   : AIC=291.927, Time=0.06 sec
 ARIMA(2,2,1)(0,0,0)[0] intercept   : AIC=278.489, Time=0.04 sec
 ARIMA(4,2,1)(0,0,0)[0] intercept   : AIC=274.925, Time=0.11 sec
 ARIMA(4,2,2)(0,0,0)[0] intercept   : AIC=inf, Time=0.23 sec
 ARIMA(3,2,2)(0,0,0)[0] intercept   : AIC=279.220, Time=0.07 sec
 ARIMA(4,2,1)(0,0,0)[0]             : AIC=272.979, Time=0.07 sec
 ARIMA(3,2,1)(0,0,0)[0]             : AIC=290.061, Time=0.15 sec
 ARIMA(4,2,0)(0,0,0)[0]             : AIC=275.313, Time=0.03 sec
 ARIMA(4,2,2)(0,0,0)[0]             : AIC=inf, Time=0.21 sec
 ARIMA(3,2,0)(0,0,0)[0]             : AIC=273.314, Time=0.03 sec
 ARIMA(3,2,2)(0,0,0)[0]          

/Users/isi/.pyenv/versions/3.9.9/envs/population-prediction/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/isi/.pyenv/versions/3.9.9/envs/population-prediction/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/isi/.pyenv/versions/3.9.9/envs/population-prediction/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/isi/.pyenv/versions/3.9.9/envs/population-prediction/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.

 ARIMA(1,2,0)(0,0,0)[0] intercept   : AIC=437.060, Time=0.04 sec
 ARIMA(0,2,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.06 sec
 ARIMA(0,2,0)(0,0,0)[0]             : AIC=619.812, Time=0.01 sec
 ARIMA(2,2,0)(0,0,0)[0] intercept   : AIC=339.053, Time=0.04 sec
 ARIMA(3,2,0)(0,0,0)[0] intercept   : AIC=323.497, Time=0.04 sec
 ARIMA(4,2,0)(0,0,0)[0] intercept   : AIC=304.742, Time=0.08 sec
 ARIMA(4,2,1)(0,0,0)[0] intercept   : AIC=306.732, Time=0.09 sec
 ARIMA(3,2,1)(0,0,0)[0] intercept   : AIC=311.120, Time=0.11 sec
 ARIMA(4,2,0)(0,0,0)[0]             : AIC=304.206, Time=0.05 sec
 ARIMA(3,2,0)(0,0,0)[0]             : AIC=322.161, Time=0.03 sec
 ARIMA(4,2,1)(0,0,0)[0]             : AIC=306.085, Time=0.06 sec
 ARIMA(3,2,1)(0,0,0)[0]             : AIC=309.861, Time=0.13 sec

Best model:  ARIMA(4,2,0)(0,0,0)[0]          
Total fit time: 0.744 seconds
Performing stepwise search to minimize aic
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=135.152, Time=0.01 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : A

...

/Users/isi/.pyenv/versions/3.9.9/envs/population-prediction/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/isi/.pyenv/versions/3.9.9/envs/population-prediction/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/isi/.pyenv/versions/3.9.9/envs/population-prediction/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/isi/.pyenv/versions/3.9.9/envs/population-prediction/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.

 ARIMA(2,2,0)(0,0,0)[0] intercept   : AIC=69.634, Time=0.11 sec
 ARIMA(3,2,0)(0,0,0)[0] intercept   : AIC=50.843, Time=0.10 sec
 ARIMA(4,2,0)(0,0,0)[0] intercept   : AIC=52.072, Time=0.15 sec
 ARIMA(3,2,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.21 sec
 ARIMA(2,2,1)(0,0,0)[0] intercept   : AIC=54.855, Time=0.11 sec
 ARIMA(4,2,1)(0,0,0)[0] intercept   : AIC=51.698, Time=0.35 sec
 ARIMA(3,2,0)(0,0,0)[0]             : AIC=50.614, Time=0.07 sec
 ARIMA(2,2,0)(0,0,0)[0]             : AIC=72.433, Time=0.05 sec
 ARIMA(4,2,0)(0,0,0)[0]             : AIC=52.130, Time=0.11 sec
 ARIMA(3,2,1)(0,0,0)[0]             : AIC=52.173, Time=0.11 sec
 ARIMA(2,2,1)(0,0,0)[0]             : AIC=56.297, Time=0.07 sec
 ARIMA(4,2,1)(0,0,0)[0]             : AIC=51.309, Time=0.37 sec

Best model:  ARIMA(3,2,0)(0,0,0)[0]          
Total fit time: 1.954 seconds
Performing stepwise search to minimize aic


/Users/isi/.pyenv/versions/3.9.9/envs/population-prediction/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/isi/.pyenv/versions/3.9.9/envs/population-prediction/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/isi/.pyenv/versions/3.9.9/envs/population-prediction/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/isi/.pyenv/versions/3.9.9/envs/population-prediction/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.

 ARIMA(0,2,0)(0,0,0)[0] intercept   : AIC=-282.619, Time=0.15 sec
 ARIMA(1,2,0)(0,0,0)[0] intercept   : AIC=-347.438, Time=0.12 sec
 ARIMA(0,2,1)(0,0,0)[0] intercept   : AIC=-319.750, Time=0.13 sec
 ARIMA(0,2,0)(0,0,0)[0]             : AIC=-284.601, Time=0.03 sec
 ARIMA(2,2,0)(0,0,0)[0] intercept   : AIC=-347.439, Time=0.24 sec
 ARIMA(3,2,0)(0,0,0)[0] intercept   : AIC=-365.519, Time=0.20 sec
 ARIMA(4,2,0)(0,0,0)[0] intercept   : AIC=-365.354, Time=0.48 sec
 ARIMA(3,2,1)(0,0,0)[0] intercept   : AIC=-363.953, Time=0.71 sec
 ARIMA(2,2,1)(0,0,0)[0] intercept   : AIC=-343.552, Time=0.32 sec
 ARIMA(4,2,1)(0,0,0)[0] intercept   : AIC=-364.671, Time=0.67 sec


KeyboardInterrupt: 

In [53]:
to_csv(final_result, f'forecast_per_country_2030-2050')